In [83]:
import numpy as np

class SlidingPuzzle():
    def __init__(self, num_blocos):
        self.num_blocos = num_blocos

    def _encontra_posicao(self, estado, elemento):
        for i in range(self.num_blocos):
            for j in range(self.num_blocos):
                if estado[i, j] == elemento:
                    return i, j
        return None, None

    def verifica_estados(self, atual, objetivo):
        flag = True
        for i in range(self.num_blocos):
            if flag == False:
                break

            for j in range(self.num_blocos):
                if atual[i, j] != objetivo[i, j]:
                    flag = False;
                    break;

        return flag

    def expande_estado(self, atual):
        novos_estados = []
        linha, coluna = self._encontra_posicao(atual, 0)

        # Cima
        if linha > 0:
            novo_estado = np.copy(atual)
            nova_linha = linha - 1

            bloco_alvo = novo_estado[nova_linha, coluna]
            novo_estado[nova_linha, coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)

        # Baixo
        if linha < self.num_blocos - 1:
            novo_estado = np.copy(atual)
            nova_linha = linha + 1

            bloco_alvo = novo_estado[nova_linha, coluna]
            novo_estado[nova_linha, coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)


        # Esquerda
        if coluna > 0:
            novo_estado = np.copy(atual)
            nova_coluna = coluna - 1

            bloco_alvo = novo_estado[linha, nova_coluna]
            novo_estado[linha, nova_coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)

        # Direita
        if coluna < self.num_blocos - 1:
            novo_estado = np.copy(atual)
            nova_coluna = coluna + 1

            bloco_alvo = novo_estado[linha, nova_coluna]
            novo_estado[linha, nova_coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)

        return novos_estados   

    def heuristica(self, atual, objetivo):
      total_dist = 0

      for i in range(self.num_blocos):
        for j in range(self.num_blocos):
          element = atual [i , j] #pegando o elemto atual na matriz
          line, column = self._encontra_posicao(target , element) # vendo a posição do elemento na matriz

          dist =  max(abs(line - i) , abs(column - j))  # abs() pra nao dar negativo
          total_dist +=dist

      return total_dist 

      #IMPLEMENTAR DISTANCIA DE PEÇAS NAS POSIÇÕES ERRADAS() a quantidade de peças erradas em relação a um estado objetivo
    def manhattanDistance(self, current, target ): # somas dos movimentos dos elemtos ate o lugar certo 
      total_dist = 0

      for i in range(self.num_blocos):
        for j in range(self.num_blocos):
          element = current [i , j] #pegando o elemto atual na matriz
          line, column = self._encontra_posicao(target , element) # vendo a posição do elemento na matriz

          dist =  abs(line - i) + abs(column - j)  # abs() pra nao dar negativo, dist do elemento ate sua posição correta
          total_dist +=dist

      return total_dist # retornando o total de distacia dos elemtos ao objetivo delas (posição correta)

In [84]:
import heapq

class Busca():
    def __init__(self, problema):
        self.problema = problema

    def _verifica_visitado(self, estado, estados_visitados):
        for i in estados_visitados:
            if self.problema.verifica_estados(i, estado):
                return True
        return False

    def busca(self, inicio, fim,Chebyshev):
        p_fila = []
        heapq.heappush(p_fila, (0, 0, inicio))

        id_estado = 0
        solucao_encontrada = False
        estados_visitados = []
        cont_estados = 0

        while not len(p_fila) == 0:
            atual = heapq.heappop(p_fila)[2]
            estados_visitados.append(atual)

            if self.problema.verifica_estados(atual, fim):
                solucao_encontrada = True
                break
            else:
                cont_estados += 1
                print("Visitando #", cont_estados)
                novos_estados = self.problema.expande_estado(atual)

                for i in novos_estados:
                    if not self._verifica_visitado(i, estados_visitados):
                        id_estado += 1
                        
                        if Chebyshev:
                          prioridade = self.problema.heuristica(i, fim)
                        else:
                          prioridade = self.problema.manhattanDistance(i, fim)
                        print(i, "\n h =", prioridade)
                        heapq.heappush(p_fila, (prioridade, id_estado, i))
        
        return solucao_encontrada, estados_visitados, cont_estados

In [89]:
problema = SlidingPuzzle(3)

start = np.matrix([[2, 8, 3], [1, 6, 4], [7, 5, 0]])
print(start, "\n************\n")

target = np.matrix([[1, 2, 3], [8, 0, 4], [7, 6, 5]])
print(target, "\n************\n")

print("Distância de Chebyshev:")
busca = Busca(problema)
Chebyshev=True
solution, visited, steps = busca.busca(start,target,Chebyshev)


if solution:
  print(f"Solucao encontrada com {steps} passos " )
else:
  print("Solucao nao encontrada")
print(f"visitados {visited}  " )

print("\n\n")
print("quantidade de elementos errados em relação ao objetivo final:")
busca2 = Busca(problema)
Chebyshev=False
solucao, visitados, passos = busca2.busca(start,target,Chebyshev)

if solucao:
  print(f"Solucao encontrada com {passos} passos " )
else:
  print("Solucao nao encontrada")

print(f"visitados {visitados}  " )



[[2 8 3]
 [1 6 4]
 [7 5 0]] 
************

[[1 2 3]
 [8 0 4]
 [7 6 5]] 
************

Distância de Chebyshev:
Visitando # 1
[[2 8 3]
 [1 6 0]
 [7 5 4]] 
 h = 7
[[2 8 3]
 [1 6 4]
 [7 0 5]] 
 h = 5
Visitando # 2
[[2 8 3]
 [1 0 4]
 [7 6 5]] 
 h = 3
[[2 8 3]
 [1 6 4]
 [0 7 5]] 
 h = 6
Visitando # 3
[[2 0 3]
 [1 8 4]
 [7 6 5]] 
 h = 4
[[2 8 3]
 [0 1 4]
 [7 6 5]] 
 h = 4
[[2 8 3]
 [1 4 0]
 [7 6 5]] 
 h = 5
Visitando # 4
[[0 2 3]
 [1 8 4]
 [7 6 5]] 
 h = 3
[[2 3 0]
 [1 8 4]
 [7 6 5]] 
 h = 5
Visitando # 5
[[1 2 3]
 [0 8 4]
 [7 6 5]] 
 h = 2
Visitando # 6
[[1 2 3]
 [7 8 4]
 [0 6 5]] 
 h = 3
[[1 2 3]
 [8 0 4]
 [7 6 5]] 
 h = 0
Solucao encontrada com 6 passos 
visitados [matrix([[2, 8, 3],
        [1, 6, 4],
        [7, 5, 0]]), array([[2, 8, 3],
       [1, 6, 4],
       [7, 0, 5]]), array([[2, 8, 3],
       [1, 0, 4],
       [7, 6, 5]]), array([[2, 0, 3],
       [1, 8, 4],
       [7, 6, 5]]), array([[0, 2, 3],
       [1, 8, 4],
       [7, 6, 5]]), array([[1, 2, 3],
       [0, 8, 4],
       [7, 